In [8]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

#from algorithms.algorithms.one_plus_one_cma import OnePlusOneCMAES
import ioh 

import importlib.util

# The path to the lumapi.py file
file_path = 'C:\\Program Files\\Lumerical\\v241\\api\\python\\lumapi.py'
spec_win = importlib.util.spec_from_file_location('lumapi', file_path)
lumapi = importlib.util.module_from_spec(spec_win)
spec_win.loader.exec_module(lumapi)

import scipy.io

In [9]:
#######   CMA-ES analysis ########

filename = 'CMA_ES_run_result.txt'

raw_y = []
matrix = []

with open(filename, 'r') as f:
    next(f)
    for line in f:
        line = line.strip()
        if not line:
            continue
        data = line.split()
        if len(data) == 13:
            raw_y.append(float(data[1]))
            row = [float(val) for val in data[2:]]
            matrix.append(row)

eff_array = np.array(raw_y)*-1
params_norm_mat = np.array(matrix)

params_mat = []

for i in range(len(params_norm_mat)):
    params_norm = params_norm_mat[i]
    widths_x_norm, widths_z_norm, H_norm = params_norm[:5], params_norm[5:10], params_norm[-1] 

    period = 2.07e-6
    widths_x = (widths_x_norm + 1) * (period/5.1 / 2)
    widths_z = (widths_z_norm + 1) * (period/5.1 / 2)
    H = (H_norm + 1) * (0.6e-6 - 0.2e-6) / 2 + 0.2e-6

    params = np.concatenate([widths_x, widths_z, [H]])
    params_mat.append(params)

params_mat = np.array(params_mat)
eff_array = eff_array

best_params_mat = params_mat[np.where(eff_array >= 0.802)[0][0]]

best_params_mat

array([1.63539741e-07, 1.79211062e-07, 1.52327850e-07, 2.08341644e-07,
       1.77976368e-07, 2.09259547e-07, 1.71454447e-07, 2.11643294e-07,
       1.74291159e-07, 1.85095341e-07, 2.00000000e-07])

In [15]:

def immersed_grating_fdtd(params, polarization, wavelength_i, polarization_angle, mesh_accuracy, run_switch=1, system_error = None):

    if system_error != None:
        params = params + system_error
        filename = 'manTol_2d_Design_field' +  f"{polarization}" + f"{wavelength_i*1e6}" +"_" + f"{system_error*1e9}" +".fsp"
    else:
        params = params
        filename = 'manTol_2d_Design_field' +  f"{polarization}" + f"{wavelength_i*1e6}" + ".fsp"
    fdtd = lumapi.FDTD(hide=True)

    fdtd.switchtolayout
    fdtd.deleteall
    #etch slab from y=0.4 to y=0.6
    #coating slab from y=0 to y=0.4 #0.4 mircrons coating width
    #gratings at y=0
    #immersion slab from y=0 to y=-0.8
    # all slabs are 1 period + some "extra" in width 

    period = 2.07e-6
    incidence_angle = 62.6
    #polarization_angle = 45
    #wavelength_i = 2.345e-6

    #addstructuregroup();
    # Add and configure the "alGrating" structure group
    fdtd.addstructuregroup()
    fdtd.set("name", "alGrating")
    fdtd.set("enabled", 1)

    fdtd.addstructuregroup()
    fdtd.set("name", "coatingSlab")
    fdtd.set("enabled", 1)

    fdtd.addstructuregroup()
    fdtd.set("name", "etchSlab")
    fdtd.set("enabled", 1)

    fdtd.addstructuregroup()
    fdtd.set("name", "immersionSlab")
    fdtd.set("enabled", 1)

    widths_x, widths_z, H = params[:5], params[5:10], params[-1] 

    ################################### Al gratings ###################################
  
    #widths_x, widths_z, H = params[:5], params[5:10], params[-1] 

    material = "Al (Aluminium) - Palik"

    period = 2.07e-6
    
    centers_z = np.array([0.5, 1.5, 2.5, 3.5, 4.5])*(period/5)
    centers_x = np.array([0.5, 1.5, 2.5, 3.5, 4.5])*(period/5)



    # Loop over widths in x-dimension
    for xi in range(len(widths_x)):
        center_x = centers_x[xi]
        width_x = widths_x[xi]
        
        # Loop over widths in z-dimension
        for zi in range(len(widths_z)):
            center_z = centers_z[zi]
            width_z = widths_z[xi]

            #print(f"{width_x} - {width_z}")
            
            # Create a rectangular pillar
            fdtd.addrect()
            fdtd.set('x', center_x)  # Center position in x
            fdtd.set('x span', width_x)  # Width in x-dimension
            fdtd.set('y', -H / 2)  # Base position in y
            fdtd.set('y span', H)  # Height in y-dimension
            fdtd.set('z', center_z)  # Center position in z
            fdtd.set('z span', width_z)  # Width in z-dimension
            
            fdtd.set('material', material)  # Set the material
            fdtd.set('name', f'grating_x{xi+1}_z{zi+1}')  # Unique name for each rectangle
            fdtd.addtogroup("alGrating")  # Add to the group


    ################################### Coating slab ###################################
    coating_material = "Al (Aluminium) - Palik"
    offset = period * 0.2  # 20% of the period

    ymin = 0
    ymax = 0.4e-6  # 0.4 microns coating width

    slabs_width = period + offset
    slabs_center = period / 2

    # Add a rectangle and set properties
    fdtd.addrect()
    fdtd.set("x", slabs_center)  # Center position in x
    fdtd.set("x span", slabs_width)  # Width in x-dimension
    fdtd.set("y min", ymin)  # Base position in y
    fdtd.set("y max", ymax)  # Height in y-dimension
    fdtd.set("z", slabs_center)  # Center position in z
    fdtd.set("z span", slabs_width)  # Width in z-dimension
    fdtd.set("material", coating_material)  # Set the material
    fdtd.set("name", "coatingSlab")
    fdtd.addtogroup("coatingSlab")  # Add to the group


    ################################### Etch slab ###################################
    coating_material = "etch"  # Assuming "etch" is a placeholder for the actual material name
    offset = period * 0.2  # 20% of the period

    ymin = 0.4e-6  # Start of etch slab in y-direction
    ymax = 0.6e-6  # End of etch slab in y-direction

    slabs_width = period + offset  # Width of the slab including offset
    slabs_center = period / 2  # Center position of the slab

    # Add a rectangle for the etch slab
    fdtd.addrect()
    fdtd.set("x", slabs_center)  # Center position in x
    fdtd.set("x span", slabs_width)  # Width in x-dimension
    fdtd.set("y min", ymin)  # Base position in y
    fdtd.set("y max", ymax)  # Height in y-dimension
    fdtd.set("z", slabs_center)  # Center position in z
    fdtd.set("z span", slabs_width)  # Width in z-dimension
    fdtd.set("material", coating_material)  # Set the material
    fdtd.set("name", "etchSlab")
    fdtd.addtogroup("etchSlab")  # Add to the group

    ################################### Immersion slab ###################################
    immersion_material = "Si (Silicon) - Palik"  # Silicon material from Palik's database
    offset = period * 0.2  # 20% of the period

    ymin = -0.8e-6  # Start of immersion slab in y-direction
    ymax = 0.0  # End of immersion slab in y-direction

    slabs_width = period + offset  # Width of the slab including offset
    slabs_center = period / 2  # Center position of the slab

    # Add a rectangle for the immersion slab
    fdtd.addrect()
    fdtd.set("x", slabs_center)  # Center position in x
    fdtd.set("x span", slabs_width)  # Width in x-dimension
    fdtd.set("y min", ymin)  # Base position in y
    fdtd.set("y max", ymax)  # Height in y-dimension
    fdtd.set("z", slabs_center)  # Center position in z
    fdtd.set("z span", slabs_width)  # Width in z-dimension

    # Mesh settings
    fdtd.set("override mesh order from material database", 1)
    fdtd.set("mesh order", 3)  # Set the mesh order explicitly

    # Set the material and other properties
    fdtd.set("material", immersion_material)  # Set the material
    fdtd.set("name", "immersionSlab")
    fdtd.addtogroup("immersionSlab")  # Add to the group


    ################################### index monitor ###################################
    offset = period * 0.05  # 5% of the period
    ymin = -0.75e-6  # Start of index monitor region in y-direction
    ymax = 0.5e-6  # End of index monitor region in y-direction

    slabs_width = period + offset  # Width of the index monitor including offset
    slabs_center = period / 2  # Center position of the index monitor

    # Add an index monitor
    fdtd.addindex()

    # Set properties for the index monitor
    fdtd.set("x", slabs_center)  # Center position in x
    fdtd.set("x span", slabs_width)  # Width in x-dimension
    fdtd.set("y min", ymin)  # Base position in y
    fdtd.set("y max", ymax)  # Height in y-dimension
    fdtd.set("z", slabs_center / 5)  # Center position in z
    # fdtd.set("z span", slabs_width)  # Width in z-dimension, commented out as in original script

    # Further monitor settings
    fdtd.set("name", "index_monitor")
    fdtd.set("monitor type", 3)  # 2D z-normal monitor type



    ################################### FDTD ###################################
    ymin = -0.75e-6; ymax = 0.5e-6; #immersion slab from y=0 to y=-0.8

    slabs_width = period
    slabs_center = slabs_width/2

    fdtd.addfdtd()
    fdtd.set("dimension", '3D')  # 3D
    fdtd.set('x',slabs_center)  # Center position in x
    fdtd.set('x span', slabs_width)  # Width in x-dimension

    fdtd.set('y min', ymin)  # Base position in y
    fdtd.set('y max',ymax)  # Height in y-dimension

    fdtd.set('z', slabs_center/5);  # Center position in z
    fdtd.set('z span', slabs_width/5)  # Width in z-dimension

    fdtd.set('x min bc', 'Bloch')
    fdtd.set('z min bc', 'Bloch')

    fdtd.set("mesh accuracy", mesh_accuracy)  # Set the mesh order explicitly #2, 15, 50, 120, 240, 460 mins


    ################################### Source ###################################
    fdtd.addplane()
    fdtd.set('injection axis', 'y')
    fdtd.set('x', period / 2)
    y_source = ymin * 0.90
    fdtd.set('y', y_source)
    fdtd.set('z', period / 5 / 2)
    fdtd.set('angle theta', incidence_angle)
    fdtd.set('angle phi', 90)
    if polarization == "U":
        fdtd.set('polarization angle', polarization_angle)
    else:
        fdtd.set("polarization definition", polarization)
    fdtd.set('plane wave type', 'Bloch/periodic')
    fdtd.set('wavelength start', wavelength_i)
    fdtd.set('wavelength stop', wavelength_i)

    ################################### reflection monitor ###################################
    offset = period * 0.5  # Additional offset
    slabs_width = period + offset
    slabs_center = period / 2

    fdtd.addpower()
    fdtd.set("name", "reflection_grating")
    fdtd.set("monitor type", "3D")
    fdtd.set('x', slabs_center)
    fdtd.set('x span', slabs_width)
    y_monitor = ymin * 0.95
    fdtd.set('y', y_monitor)
    fdtd.set('y min', y_monitor)
    fdtd.set('y max', y_monitor)
    fdtd.set('z', slabs_center / 5)
    fdtd.set('z span', slabs_width / 5)

    # Run the simulation
    fdtd.save(filename)
    index_profile = fdtd.getresult("index_monitor","index")
        
    # Run the simulation

    if run_switch == 1:
        fdtd.run()
        Tr = fdtd.transmission("reflection_grating", 1)
        
        mname = "reflection_grating"  # monitor name
        G = fdtd.grating(mname)  # power to each order (fraction of transmitted power)
        n_orders = fdtd.gratingn(mname)  # power to each order (fraction of transmitted power)
        m_orders = fdtd.gratingm(mname)  # power to each order (fraction of transmitted power)
        u1_vectors = fdtd.gratingu1(mname)
        u2_vectors = fdtd.gratingu1(mname)
        return n_orders, m_orders, u1_vectors, u2_vectors, np.array(-Tr * G), index_profile
    else:
        return np.zeros([1,2]),np.zeros([1,2]), np.zeros([1,2]),  np.zeros([1,2]), np.zeros([1,2]), index_profile



In [16]:
params = best_params_mat # best point from
params

array([1.63539741e-07, 1.79211062e-07, 1.52327850e-07, 2.08341644e-07,
       1.77976368e-07, 2.09259547e-07, 1.71454447e-07, 2.11643294e-07,
       1.74291159e-07, 1.85095341e-07, 2.00000000e-07])

In [17]:
''' 

mesh_accuracy = 2

# Define wavelengths
wavelength_arr = np.linspace(2.304e-6, 2.400e-6, 7)
drift_arr = np.linspace(-0.75e-9, +0.75e-9, 9)

for drift in drift_arr:
    for wavelength_i in wavelength_arr:
        params_drifted = params + drift
        n_orders, _, _, _, eff_wl, index_profile = immersed_grating_fdtd(params_drifted, 'U', wavelength_i, 45, mesh_accuracy)'''

" \n\nmesh_accuracy = 2\n\n# Define wavelengths\nwavelength_arr = np.linspace(2.304e-6, 2.400e-6, 7)\ndrift_arr = np.linspace(-0.75e-9, +0.75e-9, 9)\n\nfor drift in drift_arr:\n    for wavelength_i in wavelength_arr:\n        params_drifted = params + drift\n        n_orders, _, _, _, eff_wl, index_profile = immersed_grating_fdtd(params_drifted, 'U', wavelength_i, 45, mesh_accuracy)"

In [ ]:

wavelength_arr = np.array([np.linspace(2.304e-6, 2.400e-6, 7)[3]])

pol_arr = ["U"] #["S", "P", "U"]
mesh_accuracy = 4
sys_error_arr = np.array([-10e-9, 10e-9, 20e-9, -20e-9, 0, -25e-9, 25e-9])

npz_filename = "sys_error_m2_2.npz"

results = {
    "wavelength": [],
    "polarization": [],
    "efficiency": [],
    "n_orders": [],
    "m_orders": [],
    "u1_vectors": [],
    "u2_vectors": [],
    "index_profile": []
}

for sys_error in sys_error_arr:
    for wavelength_i in wavelength_arr:
        for pol in pol_arr:
            
            params_sys_error = sys_error + params

            print(params_sys_error)

            n_orders, m_orders, u1_vectors, u2_vectors, eff_val, index_profile = immersed_grating_fdtd(
                params, pol, wavelength_i, 45, mesh_accuracy, run_switch=1, system_error=sys_error)

            eff_5_val = eff_val[0][0]

            results["wavelength"].append(wavelength_i)
            results["polarization"].append(pol)
            results["efficiency"].append(eff_val)
            results["n_orders"].append(n_orders)
            results["m_orders"].append(m_orders)
            results["u1_vectors"].append(u1_vectors)
            results["u2_vectors"].append(u2_vectors)
            results["index_profile"].append(index_profile)

            #np.savez(npz_filename, **results)

            print(f"Error: {sys_error * 1e9}(nm) wavelength: {wavelength_i:.4e} m, Polarization: {pol}, eff_5_val: {eff_5_val:.4f}")

[1.53539741e-07 1.69211062e-07 1.42327850e-07 1.98341644e-07
 1.67976368e-07 1.99259547e-07 1.61454447e-07 2.01643294e-07
 1.64291159e-07 1.75095341e-07 1.90000000e-07]
Error: -10.0(nm) wavelength: 2.3040e-06 m, Polarization: U, eff_5_val: 0.7661
[1.53539741e-07 1.69211062e-07 1.42327850e-07 1.98341644e-07
 1.67976368e-07 1.99259547e-07 1.61454447e-07 2.01643294e-07
 1.64291159e-07 1.75095341e-07 1.90000000e-07]
Error: -10.0(nm) wavelength: 2.4000e-06 m, Polarization: U, eff_5_val: 0.8008
[1.73539741e-07 1.89211062e-07 1.62327850e-07 2.18341644e-07
 1.87976368e-07 2.19259547e-07 1.81454447e-07 2.21643294e-07
 1.84291159e-07 1.95095341e-07 2.10000000e-07]
Error: 10.0(nm) wavelength: 2.3040e-06 m, Polarization: U, eff_5_val: 0.7603
[1.73539741e-07 1.89211062e-07 1.62327850e-07 2.18341644e-07
 1.87976368e-07 2.19259547e-07 1.81454447e-07 2.21643294e-07
 1.84291159e-07 1.95095341e-07 2.10000000e-07]
Error: 10.0(nm) wavelength: 2.4000e-06 m, Polarization: U, eff_5_val: 0.7509
[1.83539741e-0

# Results
#

[1.38539741e-07 1.54211062e-07 1.27327850e-07 1.83341644e-07
 1.52976368e-07 1.84259547e-07 1.46454447e-07 1.86643294e-07
 1.49291159e-07 1.60095341e-07 1.75000000e-07]
Error: -25.0(nm) wavelength: 2.3520e-06 m, Polarization: U, eff_5_val: 0.6859

[1.88539741e-07 2.04211062e-07 1.77327850e-07 2.33341644e-07
 2.02976368e-07 2.34259547e-07 1.96454447e-07 2.36643294e-07
 1.99291159e-07 2.10095341e-07 2.25000000e-07]
Error: 25.0(nm) wavelength: 2.3520e-06 m, Polarization: U, eff_5_val: 0.6996

[1.63539741e-07 1.79211062e-07 1.52327850e-07 2.08341644e-07
 1.77976368e-07 2.09259547e-07 1.71454447e-07 2.11643294e-07
 1.74291159e-07 1.85095341e-07 2.00000000e-07]
Error: 0.0(nm) wavelength: 2.3520e-06 m, Polarization: U, eff_5_val: 0.7906

[1.53539741e-07 1.69211062e-07 1.42327850e-07 1.98341644e-07
 1.67976368e-07 1.99259547e-07 1.61454447e-07 2.01643294e-07
 1.64291159e-07 1.75095341e-07 1.90000000e-07]
Error: -10.0(nm) wavelength: 2.3520e-06 m, Polarization: U, eff_5_val: 0.7992

[1.73539741e-07 1.89211062e-07 1.62327850e-07 2.18341644e-07
 1.87976368e-07 2.19259547e-07 1.81454447e-07 2.21643294e-07
 1.84291159e-07 1.95095341e-07 2.10000000e-07]
Error: 10.0(nm) wavelength: 2.3520e-06 m, Polarization: U, eff_5_val: 0.7512

[1.53539741e-07 1.69211062e-07 1.42327850e-07 1.98341644e-07
 1.67976368e-07 1.99259547e-07 1.61454447e-07 2.01643294e-07
 1.64291159e-07 1.75095341e-07 1.90000000e-07]
Error: -10.0(nm) wavelength: 2.3520e-06 m, Polarization: U, eff_5_val: 0.7992

[1.73539741e-07 1.89211062e-07 1.62327850e-07 2.18341644e-07
 1.87976368e-07 2.19259547e-07 1.81454447e-07 2.21643294e-07
 1.84291159e-07 1.95095341e-07 2.10000000e-07]
Error: 10.0(nm) wavelength: 2.3520e-06 m, Polarization: U, eff_5_val: 0.7512

[1.83539741e-07 1.99211062e-07 1.72327850e-07 2.28341644e-07
 1.97976368e-07 2.29259547e-07 1.91454447e-07 2.31643294e-07
 1.94291159e-07 2.05095341e-07 2.20000000e-07]
Error: 20.0(nm) wavelength: 2.3520e-06 m, Polarization: U, eff_5_val: 0.7164

[1.43539741e-07 1.59211062e-07 1.32327850e-07 1.88341644e-07
 1.57976368e-07 1.89259547e-07 1.51454447e-07 1.91643294e-07
 1.54291159e-07 1.65095341e-07 1.80000000e-07]
Error: -20.0(nm) wavelength: 2.3520e-06 m, Polarization: U, eff_5_val: 0.7581

[1.63539741e-07 1.79211062e-07 1.52327850e-07 2.08341644e-07
 1.77976368e-07 2.09259547e-07 1.71454447e-07 2.11643294e-07
 1.74291159e-07 1.85095341e-07 2.00000000e-07]
Error: 0.0(nm) wavelength: 2.3520e-06 m, Polarization: U, eff_5_val: 0.7906

[1.38539741e-07 1.54211062e-07 1.27327850e-07 1.83341644e-07
 1.52976368e-07 1.84259547e-07 1.46454447e-07 1.86643294e-07
 1.49291159e-07 1.60095341e-07 1.75000000e-07]
Error: -25.0(nm) wavelength: 2.3520e-06 m, Polarization: U, eff_5_val: 0.6859

[1.88539741e-07 2.04211062e-07 1.77327850e-07 2.33341644e-07
 2.02976368e-07 2.34259547e-07 1.96454447e-07 2.36643294e-07
 1.99291159e-07 2.10095341e-07 2.25000000e-07]
Error: 25.0(nm) wavelength: 2.3520e-06 m, Polarization: U, eff_5_val: 0.6996

[1.53539741e-07 1.69211062e-07 1.42327850e-07 1.98341644e-07
 1.67976368e-07 1.99259547e-07 1.61454447e-07 2.01643294e-07
 1.64291159e-07 1.75095341e-07 1.90000000e-07]
Error: -10.0(nm) wavelength: 2.3040e-06 m, Polarization: U, eff_5_val: 0.7661

[1.53539741e-07 1.69211062e-07 1.42327850e-07 1.98341644e-07
 1.67976368e-07 1.99259547e-07 1.61454447e-07 2.01643294e-07
 1.64291159e-07 1.75095341e-07 1.90000000e-07]
Error: -10.0(nm) wavelength: 2.4000e-06 m, Polarization: U, eff_5_val: 0.8008

[1.73539741e-07 1.89211062e-07 1.62327850e-07 2.18341644e-07
 1.87976368e-07 2.19259547e-07 1.81454447e-07 2.21643294e-07
 1.84291159e-07 1.95095341e-07 2.10000000e-07]
Error: 10.0(nm) wavelength: 2.3040e-06 m, Polarization: U, eff_5_val: 0.7603

[1.73539741e-07 1.89211062e-07 1.62327850e-07 2.18341644e-07
 1.87976368e-07 2.19259547e-07 1.81454447e-07 2.21643294e-07
 1.84291159e-07 1.95095341e-07 2.10000000e-07]
 Error: 10.0(nm) wavelength: 2.4000e-06 m, Polarization: U, eff_5_val: 0.7509

[1.83539741e-07 1.99211062e-07 1.72327850e-07 2.28341644e-07
 1.97976368e-07 2.29259547e-07 1.91454447e-07 2.31643294e-07
 1.94291159e-07 2.05095341e-07 2.20000000e-07]
Error: 20.0(nm) wavelength: 2.3040e-06 m, Polarization: U, eff_5_val: 0.7185

[1.83539741e-07 1.99211062e-07 1.72327850e-07 2.28341644e-07
 1.97976368e-07 2.29259547e-07 1.91454447e-07 2.31643294e-07
 1.94291159e-07 2.05095341e-07 2.20000000e-07]
Error: 20.0(nm) wavelength: 2.4000e-06 m, Polarization: U, eff_5_val: 0.7109

[1.43539741e-07 1.59211062e-07 1.32327850e-07 1.88341644e-07
 1.57976368e-07 1.89259547e-07 1.51454447e-07 1.91643294e-07
 1.54291159e-07 1.65095341e-07 1.80000000e-07]
Error: -20.0(nm) wavelength: 2.3040e-06 m, Polarization: U, eff_5_val: 0.7334

[1.43539741e-07 1.59211062e-07 1.32327850e-07 1.88341644e-07
 1.57976368e-07 1.89259547e-07 1.51454447e-07 1.91643294e-07
 1.54291159e-07 1.65095341e-07 1.80000000e-07]
Error: -20.0(nm) wavelength: 2.4000e-06 m, Polarization: U, eff_5_val: 0.7742

[1.63539741e-07 1.79211062e-07 1.52327850e-07 2.08341644e-07
 1.77976368e-07 2.09259547e-07 1.71454447e-07 2.11643294e-07
 1.74291159e-07 1.85095341e-07 2.00000000e-07]
Error: 0.0(nm) wavelength: 2.3040e-06 m, Polarization: U, eff_5_val: 0.7877

[1.63539741e-07 1.79211062e-07 1.52327850e-07 2.08341644e-07
 1.77976368e-07 2.09259547e-07 1.71454447e-07 2.11643294e-07
 1.74291159e-07 1.85095341e-07 2.00000000e-07]
Error: 0.0(nm) wavelength: 2.4000e-06 m, Polarization: U, eff_5_val: 0.7911

In [ ]:
wavelength: 2.3040e-06 m, Polarization: S, eff_5_val: 0.8119
wavelength: 2.3040e-06 m, Polarization: P, eff_5_val: 0.7454
wavelength: 2.3040e-06 m, Polarization: U, eff_5_val: 0.7787
wavelength: 2.3200e-06 m, Polarization: S, eff_5_val: 0.8111
wavelength: 2.3200e-06 m, Polarization: P, eff_5_val: 0.7428
wavelength: 2.3200e-06 m, Polarization: U, eff_5_val: 0.7770
wavelength: 2.3360e-06 m, Polarization: S, eff_5_val: 0.8085
wavelength: 2.3360e-06 m, Polarization: P, eff_5_val: 0.7650
wavelength: 2.3360e-06 m, Polarization: U, eff_5_val: 0.7867
wavelength: 2.3520e-06 m, Polarization: S, eff_5_val: 0.7964
wavelength: 2.3520e-06 m, Polarization: P, eff_5_val: 0.7627
wavelength: 2.3520e-06 m, Polarization: U, eff_5_val: 0.7796
wavelength: 2.3680e-06 m, Polarization: S, eff_5_val: 0.7983
wavelength: 2.3680e-06 m, Polarization: P, eff_5_val: 0.7657
wavelength: 2.3680e-06 m, Polarization: U, eff_5_val: 0.7820
wavelength: 2.3840e-06 m, Polarization: S, eff_5_val: 0.7909
wavelength: 2.3840e-06 m, Polarization: P, eff_5_val: 0.7669
wavelength: 2.3840e-06 m, Polarization: U, eff_5_val: 0.7789
wavelength: 2.4000e-06 m, Polarization: S, eff_5_val: 0.7971
wavelength: 2.4000e-06 m, Polarization: P, eff_5_val: 0.7583
wavelength: 2.4000e-06 m, Polarization: U, eff_5_val: 0.7777